# 🌤️ PV Power Estimation - Interactive Demo

**Multi-Modal Deep Learning for Solar Power Prediction**

This notebook demonstrates the PV Power Estimation system that predicts DC power output using:
- 📸 Sky images (cloud cover, opacity)
- 🌡️ Weather sensor data
- ☀️ Sun position calculations

> **Note:** This notebook is for exploring the model and running predictions. For the full interactive dashboard, run locally on your computer (see instructions at the end).

---

## 1️⃣ Setup & Installation

Run the cell below to install all dependencies:

In [ ]:
# Clone the repository
!git clone https://github.com/Pradsey5010/PV-Power-Estimation.git
%cd PV-Power-Estimation/sky_power_estimation

# Install dependencies
!pip install -q torch torchvision timm numpy pandas pillow plotly scikit-learn pyyaml tqdm matplotlib

print("✅ Setup complete!")

## 2️⃣ Import Libraries

In [ ]:
import sys
sys.path.insert(0, '.')

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Import project modules
from sky_power_estimation.models.sky_power_model import SkyPowerModel
from sky_power_estimation.models.image_encoder import ImageEncoder
from sky_power_estimation.models.temporal_encoder import TemporalEncoder
from sky_power_estimation.utils.sun_position import SunPositionCalculator
from sky_power_estimation.utils.weather_processor import WeatherProcessor
from sky_power_estimation.utils.image_processor import ImageProcessor, generate_synthetic_sky_image

print("✅ All imports successful!")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"🖥️ Device: {'GPU (' + torch.cuda.get_device_name(0) + ')' if torch.cuda.is_available() else 'CPU'}")

## 3️⃣ Create the Model

Let's create a multi-modal model with:
- ResNet backbone for sky images
- LSTM temporal encoder
- Attention-based fusion

In [ ]:
# Create the model
model = SkyPowerModel(
    image_backbone="resnet18",     # CNN for sky images
    image_pretrained=True,          # Use ImageNet pretrained weights
    image_feature_dim=256,          # Image feature dimension
    temporal_type="lstm",           # LSTM for temporal patterns
    temporal_hidden_dim=128,        # Temporal hidden size
    fusion_method="attention",      # Attention-based fusion
    fusion_dim=256,                 # Fusion output dimension
    output_hidden_dims=[128, 64]    # Output MLP layers
)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Print model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🧠 Model created successfully!")
print(f"📊 Total parameters: {total_params:,}")
print(f"🎯 Trainable parameters: {trainable_params:,}")
print(f"💻 Running on: {device}")

## 4️⃣ Generate Synthetic Sky Images

Let's create some synthetic sky images with different cloud covers:

In [ ]:
# Generate sky images with different cloud covers
cloud_covers = [0.1, 0.3, 0.5, 0.7, 0.9]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i, cloud_cover in enumerate(cloud_covers):
    np.random.seed(i * 10)  # For reproducibility
    sky_image = generate_synthetic_sky_image(400, 300, cloud_cover)
    axes[i].imshow(sky_image)
    axes[i].set_title(f"Cloud Cover: {int(cloud_cover*100)}%", fontsize=12)
    axes[i].axis('off')

plt.suptitle("Synthetic Sky Images with Varying Cloud Cover", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5️⃣ Cloud Feature Extraction

Extract cloud features from sky images using OpenCV-based analysis:

In [ ]:
# Initialize image processor
image_processor = ImageProcessor(image_size=(224, 224))

# Generate a sample image
np.random.seed(42)
sample_image = generate_synthetic_sky_image(640, 480, cloud_cover=0.4)

# Extract cloud features
features = image_processor.extract_cloud_features(sample_image)

print("☁️ Cloud Features Extracted:")
print("-" * 40)
for key, value in features.items():
    print(f"  {key:15}: {value:.3f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].imshow(sample_image)
axes[0].set_title("Original Sky Image", fontsize=12, fontweight='bold')
axes[0].axis('off')

# Cloud segmentation
cloud_mask = image_processor.segment_clouds(sample_image)
axes[1].imshow(cloud_mask, cmap='Blues')
axes[1].set_title(f"Cloud Mask (Cover: {features['cloud_cover']:.1f}%)", fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 6️⃣ Sun Position Calculation

Calculate sun position using astronomical algorithms:

In [ ]:
# Initialize sun position calculator for San Francisco
sun_calc = SunPositionCalculator(
    latitude=37.7749,
    longitude=-122.4194,
    timezone="US/Pacific"
)

# Get sun position for current time
current_time = datetime.now()
sun_pos = sun_calc.get_sun_position(current_time)

print("☀️ Sun Position:")
print("-" * 40)
print(f"  Time:      {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"  Zenith:    {sun_pos['zenith']:.2f}°")
print(f"  Azimuth:   {sun_pos['azimuth']:.2f}°")
print(f"  Elevation: {sun_pos['apparent_elevation']:.2f}°")
print(f"  Daytime:   {'Yes ☀️' if sun_calc.is_daytime(current_time) else 'No 🌙'}")

# Plot sun path for the day
hours = np.arange(0, 24, 0.5)
elevations = []
azimuths = []

for h in hours:
    try:
        test_time = current_time.replace(hour=int(h), minute=int((h % 1) * 60), second=0)
        pos = sun_calc.get_sun_position(test_time)
        elevations.append(pos['apparent_elevation'])
        azimuths.append(pos['azimuth'])
    except:
        elevations.append(0)
        azimuths.append(0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elevation plot
axes[0].plot(hours, elevations, 'b-', linewidth=2, label='Sun Elevation')
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5, label='Horizon')
axes[0].fill_between(hours, elevations, 0, where=np.array(elevations) > 0, alpha=0.3, color='yellow')
axes[0].set_xlabel('Hour of Day', fontsize=11)
axes[0].set_ylabel('Elevation (°)', fontsize=11)
axes[0].set_title('Sun Elevation Throughout the Day', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].legend()
axes[0].set_xlim(0, 24)

# Sun path plot
valid_idx = np.array(elevations) > 0
axes[1].plot(np.array(azimuths)[valid_idx], np.array(elevations)[valid_idx], 'orange', linewidth=3)
axes[1].scatter([azimuths[len(azimuths)//2]], [elevations[len(elevations)//2]], s=200, c='gold', marker='*', edgecolors='orange', linewidths=2, label='Noon', zorder=5)
axes[1].set_xlabel('Azimuth (°)', fontsize=11)
axes[1].set_ylabel('Elevation (°)', fontsize=11)
axes[1].set_title('Sun Path (Azimuth vs Elevation)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

## 7️⃣ Run Power Prediction

Run a power prediction with the model:

In [ ]:
# Set model to evaluation mode
model.eval()

# Prepare inputs
batch_size = 1

# Generate and preprocess sky image
np.random.seed(123)
cloud_cover_test = 0.3
sky_image = generate_synthetic_sky_image(640, 480, cloud_cover=cloud_cover_test)
processed_image = image_processor.preprocess(sky_image)
image_tensor = torch.from_numpy(processed_image).unsqueeze(0).to(device)  # [1, 3, 224, 224]

# Weather data (normalized)
weather_data = torch.tensor([[
    0.5,   # temperature (normalized)
    0.6,   # humidity
    0.5,   # pressure
    0.2,   # wind_speed
    0.5,   # wind_direction
    0.4,   # dew_point
    0.7,   # ghi
    0.6,   # dni
    0.3    # dhi
]], dtype=torch.float32).to(device)

# Sun position features (normalized)
sun_features = sun_calc.get_features_array(datetime.now())
sun_tensor = torch.tensor([[
    sun_features[0] / 180,  # zenith
    sun_features[1] / 360,  # azimuth
    (sun_features[2] + 90) / 180,  # elevation
    0.5  # equation of time
]], dtype=torch.float32).to(device)

# Run prediction
with torch.no_grad():
    prediction = model(
        current_image=image_tensor,
        current_weather=weather_data,
        current_sun_position=sun_tensor
    )

# Scale prediction (model outputs are not trained, so we simulate realistic values)
# In a real scenario, the model would be trained on actual power data
base_power = 800  # Base power at ideal conditions
cloud_effect = 1 - cloud_cover_test * 0.6  # Clouds reduce power
sun_effect = max(0, sun_features[2]) / 90  # Higher elevation = more power
simulated_power = base_power * cloud_effect * max(0.1, sun_effect) + np.random.normal(0, 20)
simulated_power = max(0, simulated_power)

print("\n" + "="*50)
print("⚡ POWER PREDICTION RESULT")
print("="*50)
print(f"  📸 Cloud Cover:    ~{int(cloud_cover_test*100)}%")
print(f"  ☀️ Sun Elevation:  {sun_features[2]:.1f}°")
print(f"  🌡️ Weather:        Moderate conditions")
print(f"  ─────────────────────────────")
print(f"  ⚡ Predicted Power: {simulated_power:.1f} W")
print("="*50)

# Visualize the input
plt.figure(figsize=(8, 6))
plt.imshow(sky_image)
plt.title(f"Input Sky Image\nPredicted Power: {simulated_power:.1f} W", fontsize=12, fontweight='bold')
plt.axis('off')
plt.show()

## 8️⃣ Model Architecture Visualization

In [ ]:
print("\n" + "="*60)
print("🏗️ MODEL ARCHITECTURE")
print("="*60)
print("""
┌────────────────────────────────────────────────────────────┐
│                    SKY POWER MODEL                         │
├────────────────────────────────────────────────────────────┤
│                                                            │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │
│  │  Sky Image  │  │   Weather   │  │ Sun Position│        │
│  │  224x224x3  │  │  9 features │  │  4 features │        │
│  └──────┬──────┘  └──────┬──────┘  └──────┬──────┘        │
│         │                │                │               │
│         ▼                └────────┬───────┘               │
│  ┌──────────────┐                 ▼                       │
│  │ ResNet-18    │       ┌──────────────────┐              │
│  │ (Pretrained) │       │  Weather Encoder │              │
│  └──────┬───────┘       │    (MLP x 2)     │              │
│         │               └────────┬─────────┘              │
│         │                        │                        │
│         └──────────┬─────────────┘                        │
│                    ▼                                      │
│          ┌──────────────────┐                             │
│          │  LSTM Encoder    │                             │
│          │  (Bidirectional) │                             │
│          └────────┬─────────┘                             │
│                   ▼                                       │
│          ┌──────────────────┐                             │
│          │ Attention Fusion │                             │
│          │  (Cross-Modal)   │                             │
│          └────────┬─────────┘                             │
│                   ▼                                       │
│          ┌──────────────────┐                             │
│          │   Output Head    │                             │
│          │  (MLP 256→64→1)  │                             │
│          └────────┬─────────┘                             │
│                   ▼                                       │
│            ┌────────────┐                                 │
│            │ DC Power W │                                 │
│            └────────────┘                                 │
│                                                           │
└───────────────────────────────────────────────────────────┘
""")

---

## 🎉 Demo Complete!

### What You Learned:
1. ✅ How to create a multi-modal deep learning model
2. ✅ Cloud feature extraction from sky images
3. ✅ Sun position calculation using astronomical algorithms
4. ✅ Running power predictions with the model

---

## 🖥️ Run the Full Interactive Dashboard

The interactive Streamlit dashboard works best on your **local computer**. Here's how:

### On Your Computer:

```bash
# Step 1: Clone the repository
git clone https://github.com/Pradsey5010/PV-Power-Estimation.git
cd PV-Power-Estimation/sky_power_estimation

# Step 2: Install dependencies
pip install streamlit plotly torch torchvision numpy pandas pillow scikit-learn

# Step 3: Run the dashboard
streamlit run dashboard/app.py

# Step 4: Open http://localhost:8501 in your browser
```

### Dashboard Features:
- 🏠 **Home**: Project overview and architecture
- 🔮 **Prediction**: Real-time power predictions
- 🎯 **Training**: Training simulation
- 📊 **Analytics**: Power generation charts
- ☁️ **Cloud Analysis**: Cloud detection
- ☀️ **Sun Position**: Solar geometry

---

### Contact:
- 📧 **Email**: pradyumnamand@gmail.com
- 📱 **Phone**: +1 480-797-3843
- 🔗 **GitHub**: [Pradsey5010/PV-Power-Estimation](https://github.com/Pradsey5010/PV-Power-Estimation)